In [4]:
import gurobipy as grb
import numpy as np
import pandas as pd

In [69]:
def initImport(rd):
    
    global has_repeated
    has_repeated = {}

    has_repeated["same_do-opp"] = {}
    has_repeated["opp_do-same"] = {}
    has_repeated["opp_dt-opp"] = {}
    has_repeated["opp_do-opp"] = {}


    global conflicts
    conflicts = {}

    global teams
    teams = []
    global sides
    sides = {}
    global dts #dts = Destination Teams
    dts = {}
    global dos #dos = Destination Oppositions
    dos = {}
    data_csv = pd.read_csv("data_afterr{}.csv".format(rd),index_col=0)
    conflicts_csv = pd.read_csv("conflicts.csv",index_col=0)

    # Create empty conflict dictionaries
    for ind,row in data_csv.iterrows():
        teams.append(ind)
        sides[ind] = {}
        sides[ind][1] = row[1]
        sides[ind][2] = row[4]
        sides[ind][3] = row[7]
    
        dts[ind] = {}
        dts[ind][1] = row[2]
        dts[ind][2] = row[5]
        dts[ind][3] = row[8]
    
        dos[ind] = {}
        dos[ind][1] = row[3]
        dos[ind][2] = row[6]
        dos[ind][3] = row[9]
    
    
    
        conflicts["{}_p".format(ind)] = []
        conflicts["{}_d".format(ind)] = []

    for ind,row in conflicts_csv.iterrows():
        team1 = ind
        team2 = row[0]
        
        if (team1 != "nan") and (team2 != "nan"):
            
            #1p, 2p
            if "{}_p".format(team2) not in conflicts["{}_p".format(team1)]:
                conflicts["{}_p".format(team1)].append(team2)
            #1d, 2p
            if "{}_p".format(team2) not in conflicts["{}_d".format(team1)]:
                conflicts["{}_d".format(team1)].append(team2)
            #1p, 2d
            if "{}_d".format(team2) not in conflicts["{}_p".format(team1)]:
                conflicts["{}_p".format(team1)].append(team2)
            #1d, 2d
            if "{}_d".format(team2) not in conflicts["{}_d".format(team1)]:
                conflicts["{}_d".format(team1)].append(team2)
            
            #2p, 1p
            if "{}_p".format(team1) not in conflicts["{}_p".format(team2)]:
                conflicts["{}_p".format(team2)].append(team1)
            #2d, 1p
            if "{}_p".format(team1) not in conflicts["{}_d".format(team2)]:
                conflicts["{}_d".format(team2)].append(team1)
            #2p, 1d
            if "{}_d".format(team1) not in conflicts["{}_p".format(team2)]:
                conflicts["{}_p".format(team2)].append(team1)
            #2d, 1d
            if "{}_d".format(team1) not in conflicts["{}_d".format(team2)]:
                conflicts["{}_d".format(team2)].append(team1)

def addConflicts(rd):
    for team in teams:
        
        if rd >= 2:
            # Destination Team
            if dts[team][1] not in conflicts["{}_{}".format(team,sides[team][1])]:
                conflicts["{}_{}".format(team,sides[team][1])].append(dts[team][1])
                    
            # Destination's Opponent
            if dos[team][1] not in conflicts["{}_{}".format(team,sides[team][1])]:
                conflicts["{}_{}".format(team,sides[team][1])].append(dos[team][1])
            
        if rd >= 3:
            # Destination Team
            if dts[team][2] not in conflicts["{}_{}".format(team,sides[team][2])]:
                conflicts["{}_{}".format(team,sides[team][2])].append(dts[team][2])
                    
            # Destination's Opponent
            if dos[team][2] not in conflicts["{}_{}".format(team,sides[team][2])]:
                conflicts["{}_{}".format(team,sides[team][2])].append(dos[team][2])
     
        if rd >= 4:
            # Destination Team
            if dts[team][3] not in conflicts["{}_{}".format(team,sides[team][3])]:
                conflicts["{}_{}".format(team,sides[team][3])].append(dts[team][3])
                    
            # Destination's Opponent
            if dos[team][3] not in conflicts["{}_{}".format(team,sides[team][3])]:
                conflicts["{}_{}".format(team,sides[team][3])].append(dos[team][3])
    print(conflicts)
            
def generateCosts(rd):
    global masterpairings
    global pairings
    global roomranks
    global pairingcosts
    global powercosts
    global cannotgoto
    global sendtorooms
    global receivefromrooms
    
    
   
    cannotgoto = {}
    sendtorooms = {}
    receivefromrooms = {}
    
    masterpairings = {}
    pairings = {}
    pairings[1] = {}
    pairings[2] = {}
    roomranks = {}
    pairingcosts = {}
    mastercosts = {}
    
    softconflictcosts = {}
    powercosts = {}
    venuesbyteam = {}
    
    
    div1pairings_csv = pd.read_csv("pairingsr{}_div1.csv".format(rd),index_col=0)
    div2pairings_csv = pd.read_csv("pairingsr{}_div2.csv".format(rd),index_col=0)
    
    for ind,row in div1pairings_csv.iterrows():
        pairings[1][row[0]] = [row[1],row[2]]
        masterpairings[row[0]] = [row[1],row[2]]
        roomranks[row[0]] = ind
        venuesbyteam[row[1]] = row[0]
        venuesbyteam[row[2]] = row[0]
        cannotgoto[row[1]] = []
        cannotgoto[row[2]] = []
        sendtorooms[row[0]] = []
        receivefromrooms[row[0]] = []
    
    for ind,row in div2pairings_csv.iterrows():
        pairings[2][row[0]] = [row[1],row[2]]
        masterpairings[row[0]] = [row[1],row[2]]
        roomranks[row[0]] = ind
        venuesbyteam[row[1]] = row[0]
        venuesbyteam[row[2]] = row[0]
        cannotgoto[row[1]] = []
        cannotgoto[row[2]] = []
        sendtorooms[row[0]] = []
        receivefromrooms[row[0]] = []

    #print(masterpairings)
    
    div1pairings = {}
    div2pairings = {}
    global venuenames
    venuenames = []
    
    for div1room,div1pairing in pairings[1].iteritems():
        pairingcosts["{}_notflipped".format(div1room)] = 0
        pairingcosts["{}_flipped".format(div1room)] = 0
        venuenames.append(div1room)
        div1pairings[div1room] = div1pairing
    for div2room,div2pairing in pairings[2].iteritems():
        pairingcosts["{}_notflipped".format(div2room)] = 0
        pairingcosts["{}_flipped".format(div2room)] = 0
        div2pairings[div2room] = div2pairing
        venuenames.append(div2room)
        
        
    
    for div1room,div1pairing in pairings[1].iteritems():
        
    # Unflipped
        p_nogo = []
        d_nogo = []
        pteam = div1pairing[0]
        dteam = div1pairing[1]
        # P team no-go rooms
        for conflict in conflicts["{}_p".format(pteam)]:
            p_nogo.append(venuesbyteam[conflict])
        
        for conflict in conflicts["{}_d".format(dteam)]:
            d_nogo.append(venuesbyteam[conflict])
            
        print(p_nogo)
        print(d_nogo)
        
        # 2 rooms max
        number = 0
        for room in p_nogo:
            if room in d_nogo:
                number += 1
                
        print(number)
        
        transformed = number*50
        print(transformed)
        
        if transformed == 0:
            pairingcosts["{}_notflipped".format(div1room)] = 1
        else:
            pairingcosts["{}_notflipped".format(div1room)] = transformed
        
                
        print("--")
        
        
        
    #Flipped 
        p_nogo = []
        d_nogo = []
        pteam = div1pairing[1]
        dteam = div1pairing[0]
        # P team no-go rooms
        for conflict in conflicts["{}_p".format(pteam)]:
            p_nogo.append(venuesbyteam[conflict])
        
        for conflict in conflicts["{}_d".format(dteam)]:
            d_nogo.append(venuesbyteam[conflict])
            
        print(p_nogo)
        print(d_nogo)
        
        # 2 rooms max
        number = 0
        for room in p_nogo:
            if room in d_nogo:
                number += 1
                
        print(number)
        
        transformed = number*50
        print(transformed)
        
        
        pairingcosts["{}_flipped".format(div1room)] = transformed
        print("--")
        
    
    
    
    
    
    
    for div2room,div2pairing in pairings[2].iteritems():   
    # Unflipped
        p_nogo = []
        d_nogo = []
        pteam = div2pairing[0]
        dteam = div2pairing[1]
        # P team no-go rooms
        for conflict in conflicts["{}_p".format(pteam)]:
            p_nogo.append(venuesbyteam[conflict])
        
        for conflict in conflicts["{}_d".format(dteam)]:
            d_nogo.append(venuesbyteam[conflict])
            
        print(p_nogo)
        print(d_nogo)
        
        # 2 rooms max
        number = 0
        for room in p_nogo:
            if room in d_nogo:
                number += 1
                
        print(number)
        
        transformed = number*50
        print(transformed)
        
        if transformed == 0:
            pairingcosts["{}_notflipped".format(div2room)] = 1
        else:
            pairingcosts["{}_notflipped".format(div2room)] = transformed
        
                
        print("--")
        
        
        
    #Flipped 
        p_nogo = []
        d_nogo = []
        pteam = div2pairing[1]
        dteam = div2pairing[0]
        # P team no-go rooms
        for conflict in conflicts["{}_p".format(pteam)]:
            p_nogo.append(venuesbyteam[conflict])
        
        for conflict in conflicts["{}_d".format(dteam)]:
            d_nogo.append(venuesbyteam[conflict])
            
        print(p_nogo)
        print(d_nogo)
        
        # 2 rooms max
        number = 0
        for room in p_nogo:
            if room in d_nogo:
                number += 1
                
        print(number)
        transformed = number*50
        print(transformed)
        
        
        pairingcosts["{}_flipped".format(div2room)] = transformed
        print("--")
    
    print(pairingcosts)
    print(len(pairingcosts))
    
    
# Get empty costs for each room, flipped and unflipped
# For each flipped and unflipped room, figure out what rooms each team can and can't go to - The % overlap is Part 1 of the cost
# Form combinations of flipped/unflipped (for each division)
# In each combination, get an array of all p and d teams - Create flipped arrays for Round 4
# For each team in the Round 4 array, iterate through every possible pairing in the other division
# -- a) Calculate the probability of that pairing occurring
# -- b) For that pairing, iterate through all the possible oppositions of the original team -
# -- c) Calculate the percentage overlap for that pair of pairings
# -- Expected overlap for that team in this combination is a x b x c
# Sum the overlaps from both rounds - That's the cost for that combination


# Probability of the matchup x the probability of the opposing matchup happening x the percentage overlap

# Phase 1 - Round 3 only
# Phase 2 - Round 4 only

In [73]:
m = grb.Model("Model")

x = {}

for key,value in pairingcosts.iteritems():
    x[key] = m.addVar(vtype = grb.GRB.BINARY, obj = value, name = "x_{}".format(key))
    
for venue in venuenames:
    m.addConstr(x["{}_flipped".format(venue)]+x["{}_notflipped".format(venue)] == 1)

m.modelSense = grb.GRB.MAXIMIZE

m.optimize()
    
    


for key,value in pairingcosts.iteritems():
    if x[key].x != 0:
        print(key)
    


Optimize a model with 10 rows, 20 columns and 20 nonzeros
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 251.0000000
Presolve removed 10 rows and 20 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.04 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 504 251 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.040000000000e+02, best bound 5.040000000000e+02, gap 0.0000%
H_notflipped
E_flipped
F_notflipped
D_notflipped
I_notflipped
B_notflipped
C_notflipped
A_notflipped
J_flipped
G_notflipped


In [71]:



roundNumber = 3
initImport(roundNumber)
addConflicts(roundNumber)
generateCosts(roundNumber)

{'Samios_p': ['Kelly', 'Smith'], 'Cepeda_p': ['Weerasekera', 'Pym'], 'Samios_d': ['Fast', 'Felder'], 'Smith_d': ['Franklin', 'Samios'], 'Cepeda_d': ['Self', 'Franklin'], 'Weerasekera_p': ['Burdette', 'Burdette', 'Burdette', 'Burdette', 'Cepeda', 'Stebbins'], 'Matarrese_d': ['Benjamin', 'Burdette'], 'Burdette_d': ['Weerasekera', 'Weerasekera', 'Weerasekera', 'Weerasekera', 'Gossett', 'Self'], 'Rosier_p': ['Felder', 'Fast'], 'Gossett_d': ['Burdette', 'Shapiro'], 'Burdette_p': ['Weerasekera', 'Weerasekera', 'Weerasekera', 'Weerasekera', 'Gossett', 'Matarrese'], 'Rosier_d': ['Felder', 'Benjamin'], 'Weerasekera_d': ['Burdette', 'Burdette', 'Burdette', 'Burdette', 'Kelly', 'Abry'], 'Matarrese_p': ['Fast', 'Abry'], 'Abry_p': ['Hahn', 'Weerasekera'], 'Pym_p': ['Stebbins', 'Shapiro'], 'Franklin_d': ['Smith', 'Kelly'], 'Abry_d': ['Hahn', 'Matarrese'], 'Hahn_p': ['Abry', 'Kelly'], 'Stebbins_d': ['Pym', 'Weerasekera'], 'Franklin_p': ['Smith', 'Cepeda'], 'Benjamin_p': ['Bell-Colpack', 'Rosier'], 'P